# Demo: sample crawl

In [1]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
import unidecode
from urllib.parse import unquote

PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.close()

In [2]:
whoscored_url=[
    "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
    "https://1xbet.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
    "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga",
    "https://1xbet.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
    "https://1xbet.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
]

In [3]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(whoscored_url[0])

teams_url_raw = \
driver.find_elements("xpath", "(//table[@class='grid with-centered-columns hover']/tbody/tr/td/a[@class='team-link '])")
teams_url = []

for team_url_raw in teams_url_raw:
    teams_url.append(team_url_raw.get_attribute('href'))
driver.close()
teams_url

['https://1xbet.whoscored.com/Teams/167/Show/England-Manchester-City',
 'https://1xbet.whoscored.com/Teams/26/Show/England-Liverpool',
 'https://1xbet.whoscored.com/Teams/15/Show/England-Chelsea',
 'https://1xbet.whoscored.com/Teams/30/Show/England-Tottenham',
 'https://1xbet.whoscored.com/Teams/13/Show/England-Arsenal',
 'https://1xbet.whoscored.com/Teams/32/Show/England-Manchester-United',
 'https://1xbet.whoscored.com/Teams/29/Show/England-West-Ham',
 'https://1xbet.whoscored.com/Teams/14/Show/England-Leicester',
 'https://1xbet.whoscored.com/Teams/211/Show/England-Brighton',
 'https://1xbet.whoscored.com/Teams/161/Show/England-Wolverhampton',
 'https://1xbet.whoscored.com/Teams/23/Show/England-Newcastle',
 'https://1xbet.whoscored.com/Teams/162/Show/England-Crystal-Palace',
 'https://1xbet.whoscored.com/Teams/189/Show/England-Brentford',
 'https://1xbet.whoscored.com/Teams/24/Show/England-Aston-Villa',
 'https://1xbet.whoscored.com/Teams/18/Show/England-Southampton',
 'https://1xbe

In [4]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(teams_url[1])

players_url_raw = driver.find_elements("xpath","//tbody[@id='player-table-statistics-body']/tr/td/a[@class='player-link']")
players_url = []

for player_url_raw in players_url_raw:
    players_url.append(unidecode.unidecode(unquote(player_url_raw.get_attribute('href').lower().replace('show','history'))))
driver.close()
players_url

KeyboardInterrupt: 

## tabular extract

In [ ]:
linkz = players_url[0]
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
driver.get(linkz)

#table structure
cols = driver.find_elements('xpath',"//table[@id='top-player-stats-summary-grid']/thead/tr/th")[2:]

for i in range(len(cols)):
    cols[i] = cols[i].get_attribute('innerHTML')
    

data =  driver.find_elements('xpath',"//tbody[@id='player-table-statistics-body']/tr")

pd_appender = []
for dt in data[:-1]:
    dtnr = {'name':linkz[linkz.find('history/')+8:]}
    dtnr.update(zip(cols,dt.text.split()))
    pd_appender.append(dtnr)

driver.close()


In [ ]:
e_table = pd.DataFrame(pd_appender)
e_table

## apply data type for columns & grouping

In [ ]:
e_table['Season'] = e_table['Season'].apply(lambda ss:ss[:4])
e_table['Apps'] = e_table['Apps'].apply\
(lambda apps: int(apps[:apps.find('(')])+int(apps[apps.find('(')+1:apps.find(')')]) if not apps.isdigit() else int(apps))
e_table['Mins'] = e_table['Mins'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Goals'] = e_table['Goals'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Assists'] = e_table['Assists'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Yel'] = e_table['Yel'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['MotM'] = e_table['MotM'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Red'] = e_table['Red'] .apply(lambda s: int(s) if s.isdigit() else 0)
e_table['Rating'] = e_table['Rating'] .apply(lambda s: float(s) if s != '-' else 0)

In [ ]:
e_table

In [ ]:
e_table = e_table.iloc[:,[0,1,4,5,6,7,8,9,-2,-1]]
e_table

In [ ]:
def func_agg(x):
    d = []
    for col in (list(e_table)[2:-1]):
        d.append(x[col].sum())
    d.append((x['Rating']*x['Apps']).sum()/(x['Apps'].sum()))
    return pd.Series(d, index = list(e_table)[2:])

In [ ]:
test = e_table.groupby(["name","Season"],as_index = False).apply(func_agg)
for col in list(test)[2:-1]:
    test[col] = test[col].astype('int16')
    print (col, test[col].dtype)
test['Rating'] = test['Rating'].apply(lambda rt: float("%.2f" %rt))
test

# Final crawl

In [2]:
whoscored_url=[
    "https://1xbet.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
    "https://1xbet.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
    "https://1xbet.whoscored.com/Regions/206/Tournaments/4/Spain-LaLiga",
    "https://1xbet.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
    "https://1xbet.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1"
]

In [3]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)

teams_url = []

for url in whoscored_url:
    driver.get(url)

    teams_url_raw =\
    driver.find_elements("xpath", "(//table[@class='grid with-centered-columns hover']/tbody/tr/td/a[@class='team-link '])")


    for team_url_raw in teams_url_raw:
        teams_url.append(team_url_raw.get_attribute('href'))
driver.close()
len(teams_url)

98

In [33]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)

players_url = []
for team_url in teams_url:
    driver.get(team_url)
    time.sleep(5)
    players_url_raw = driver.find_elements("xpath","//tbody[@id='player-table-statistics-body']/tr/td/a[@class='player-link']")

    for player_url_raw in players_url_raw:
        #rlink = 
        #if rlink not in players_url:
        players_url.append(unidecode.unidecode(unquote(player_url_raw.get_attribute('href').lower().replace('show','history'))))

driver.close()
len(players_url)


2926

In [35]:
len(set(players_url))

2795

In [23]:
2926, 2694

(2926, 2694)

In [ ]:
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

driver = webdriver.Chrome(service = PATH)
pd_appender = []
#table structure
linkz = players_url[0]
driver.get(linkz)


cols = driver.find_elements('xpath',"//table[@id='top-player-stats-summary-grid']/thead/tr/th")[2:]

for i in range(len(cols)):
    cols[i] = cols[i].get_attribute('innerHTML')

#crawl
for linkz in players_url:
    driver.get(linkz)
    
    data =  driver.find_elements('xpath',"//tbody[@id='player-table-statistics-body']/tr")

    for dt in data[:-1]:
        dtnr = {'name':linkz[linkz.find('history/')+8:]}
        dtnr.update(zip(cols,dt.text.split()))
        pd_appender.append(dtnr)

driver.close()

In [ ]:
e_table = pd.DataFrame(pd_appender)
e_table